跑code的要求是这样的，根据以下几个事情进行排列组合，每种情况都跑一遍，看看结果。1）a -- demean w1, b -- demean w2; 2) a -- 对于之前生成的csv文件100行 取使得n of inequality为max的beta值; b -- 100行的average (n of inequality 也是average）。 CI 用同样的方式（即如果beta是 arg max 那CI也是用arg max， 如果beta是mean, CI 也是mean）；3）bound：+-100; +-500； +-1000

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time
np.set_printoptions(suppress = True)

# Demean

df_w1
- var1 = Relationship_Dum(demean)
- var2 = mi_to_zcta5(demean)
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)


df_w2
- var1 = Relationship_Dum
- var2 = mi_to_zcta5
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)

In [ ]:
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-5].append([df1.columns[-5:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:6]+'2m').append(df2.columns[6:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:6]+'2um').append(df3.columns[6:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 5):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep

In [ ]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        # df_matched_sample = Subsample(df_matched, ratio)
        # df_unmatched_sample = Subsample(df_unmatched, ratio)
        
        loan_id = df_unmatched["loan_id"].unique()
        sample_loan_id = np.random.choice(loan_id, round(loan_id.shape[0] * ratio), replace = False)
        df_unmatched_sample = df_unmatched.loc[df_unmatched["loan_id"].isin(sample_loan_id)]
        df_matched_sample = df_matched.loc[df_matched["loan_id"].isin(sample_loan_id)]
       
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
        result = differential_evolution(objectfunc, bounds)
        
        r = np.append(result.x, round(-result.fun))
        
        df_result = df_result.append([list(r)], ignore_index = True)
        # print(i)
    df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]
    
    num_mean = df_reuslt.mean()
    num_max = df_result["Number of of inequalities satisfied"].max()
    df_result[df_result["Number of of inequalities satisfied"] == num_max]
    df_test = df_result
    df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
    df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
    
    
    '''
    print("The 5% quantile of parameters are")
    print(df_result.quantile(0.05))
    print("The 95% quantile of parameters are")
    print(df_result.quantile(0.95))
    '''
    
    return df_test

'''
t1 = time.time()
df_result = Simulation(10, 0.1, df_matched, df_unmatched)
t2 = time.time()
print("Simulation time: ", t2-t1)
df_result
'''

In [ ]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_matched = pd.read_stata('sample_matched_NY_demean1.dta').iloc[:,: 7]
df_unmatched = pd.read_stata('sample_unmatched_NY_demean1.dta').iloc[:,: 7]

df_sample = df_w1
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]

df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2021].iloc[:,: 6]

df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2021].iloc[:,: 6]





df_exchange_pairs = Exchange_pairs(df_matched, df_unmatched)

def Fox_func(num, ratio):
    df_sample = df_w1
    df_matched = df_sample[df_sample["match"] == 1]
    df_unmatched = df_sample[df_sample["match"] == 0]
    df_matched.drop(["match"], axis = 1, inplace = True) 
    df_unmatched.drop(["match"], axis = 1, inplace = True) 
    df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
    df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
    df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
    
    df_keep = Exchange_pairs(df_matched, df_unmatched)
    
    # df_keep.drop(["value3"], axis = 1, inplace = True) 
    # df_keep.drop(["value4"], axis = 1, inplace = True) 
    
    # df_keep.drop(["value2"], axis = 1) 
    # df_keep.drop(["value3"], axis = 1) 

    def objectfunc(beta, df = df_keep):
        return -sum(df.dot(beta) >=0 )
    t1 = time.time()
    # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
    # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
    bounds = [(-500, 500), (-500, -500), (-500, 500), (-500, 500)]
    # bounds = [(-500, 500), (-500, 500), (-500, 500)]
    result = differential_evolution(objectfunc, bounds)
    # print(result)

    r = np.append(result.x, round(-result.fun))
    
    # t2 = time.time()
    # print("Differential Evolution time: ", t2 - t1)
    # print("The number of inequalities satisfied is")
    # print(round(-result.fun))
    # Simulation(num, ratio, df_matched, df_unmatched)
    return r

df_result = pd.DataFrame()
for i in range(0, 100):
    print(i)
    result = Fox_func(100, 0.1)
    df_result = df_result.append([list(result)], ignore_index = True)
df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]


In [ ]:
# Don't use argmax or idxmax since if the maximum is achieved in multiple locations, only the first is returned.
num_mean = df_reuslt.mean()
num_max = df_result["Number of of inequalities satisfied"].max()
df_result[df_result["Number of of inequalities satisfied"] == num_max]
df_test = df_result
df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
df_test.to_csv("NY2020;0105.csv")